In [1]:
import numpy as np

import torch
import torch.nn as nn

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# 0. prepare data
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape
n_samples, n_features

(569, 30)

In [3]:
# split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=1234)

# scale
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455, 1]),
 torch.Size([114, 1]))

In [4]:
# 1. build model

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression(n_features)
model

LogisticRegression(
  (linear): Linear(in_features=30, out_features=1, bias=True)
)

In [5]:
# 2. loss & optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
# 3. training_loog

num_epochs = 100
for epoch in range(num_epochs):
    # forward
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    
    # backward
    loss.backward()
    
    # update
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'[epoch {epoch+1:3d}] loss : {loss.item():.4f}')

[epoch  10] loss : 0.5674
[epoch  20] loss : 0.4752
[epoch  30] loss : 0.4138
[epoch  40] loss : 0.3701
[epoch  50] loss : 0.3374
[epoch  60] loss : 0.3119
[epoch  70] loss : 0.2914
[epoch  80] loss : 0.2745
[epoch  90] loss : 0.2603
[epoch 100] loss : 0.2481


In [7]:
with torch.no_grad():
    y_pred = model(x_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    
    print(f'accuracy : {acc:.4f}')

accuracy : 0.8860
